In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
chat = ChatOpenAI()

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert. And you only reply in {language}"),
    ("ai","Ciao, mi chiano {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Sorcrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.500 χιλιόμετρα.')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text): 
        items = text.strip().split(",")
        return list(map(str.strip,items))
    

p = CommaOutputParser()

p.parse("Hello,how,are,you")
    


['Hello', 'how', 'are', 'you']

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
chat = ChatOpenAI()

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 9,900 miles (15,900 kilometers) when measured in a straight line. However, the actual distance can vary depending on the specific locations within each country.'

In [24]:
template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. DO NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colours?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'purple',
 'orange',
 'pink',
 'black',
 'white',
 'brown']

In [25]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)


examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country":"Germany"
})


KeyError: 'country'